In [149]:
import pandas as pd
from sqlalchemy import create_engine
import os
import matplotlib.pyplot as plt
import re
import tqdm

%matplotlib inline

In [134]:
data_dir = "..\\Dados"
db_file = 'fundos.db'
e = create_engine("sqlite:///"+os.path.abspath(db_file))

In [148]:
cnpjs = ['06.964.937/0001-63',
'15.831.948/0001-66',
'73.232.530/0001-39',
'12.055.107/0001-16',
'05.006.152/0001-15',
'11.188.572/0001-62',
'08.935.128/0001-59',
'15.174.615/0001-01',
'13.294.268/0001-25',
'14.138.786/0001-12',
'11.977.607/0001-42',
'13.950.080/0001-98',
'08.927.452/0001-25',
'11.435.298/0001-89',
'11.228.490/0001-02',
'11.046.362/0001-30',
'11.176.045/0001-38',
'09.412.648/0001-40',
'14.781.411/0001-76',
'08.323.402/0001-39',
'13.549.298/0001-35',
'09.635.130/0001-75']

In [151]:
cnpj = '06.964.937/0001-63' ## Kapitalo Master 1
for cnpj in tqdm.tqdm_notebook(cnpjs):
    sql_cda = "SELECT * from cda where CNPJ_FUNDO = '{}' and FILE != 'PL'".format(cnpj)
    carteira = pd.read_sql_query(sql_cda, e, parse_dates=['DT_COMPTC','DT_CONFID_APLIC','DT_EMISSAO','DT_FIM_VIGENCIA','DT_INI_VIGENCIA','DT_RISCO','DT_VENC'])


    sql_pl = "SELECT * from cda where CNPJ_FUNDO = '{}' and FILE = 'PL'".format(cnpj)
    pl = pd.read_sql_query(sql_pl, e, parse_dates=['DT_COMPTC','DT_CONFID_APLIC','DT_EMISSAO','DT_FIM_VIGENCIA','DT_INI_VIGENCIA','DT_RISCO','DT_VENC'])

    carteira['PL'] =  carteira.merge(pl[['DT_COMPTC','VL_PATRIM_LIQ']], on="DT_COMPTC", how='left')['VL_PATRIM_LIQ_y']

    name = cnpj.replace('.','').replace('/','').replace('-','') + '_' + carteira['DENOM_SOCIAL'].unique()[0].replace(' ','_') + '.xlsx'
    xlsw = pd.ExcelWriter(name)
    carteira.to_excel(xlsw, 'ativos')
    xlsw.save()

In [136]:

#carteira.loc[carteira.DS_ATIVO.str.lower().str.contains('pagar').fillna(False),'VL_MERC_POS_FINAL'] *= -1
#carteira['peso'] = carteira['VL_MERC_POS_FINAL'] / carteira['PL']

In [145]:
['06.964.937/0001-63',
'15.831.948/0001-66',
'73.232.530/0001-39',
'12.055.107/0001-16',
'05.006.152/0001-15',
'11.188.572/0001-62',
'08.935.128/0001-59',
'15.174.615/0001-01',
'13.294.268/0001-25',
'14.138.786/0001-12',
'11.977.607/0001-42',
'13.950.080/0001-98',
'08.927.452/0001-25',
'11.435.298/0001-89',
'11.228.490/0001-02',
'11.046.362/0001-30',
'11.176.045/0001-38',
'09.412.648/0001-40',
'14.781.411/0001-76',
'08.323.402/0001-39',
'13.549.298/0001-35',
'09.635.130/0001-75']

In [130]:
#coluna de id
pivot_cols = carteira.CD_ATIVO\
            .fillna(carteira.CD_ISIN)\
            .fillna(carteira[carteira.FILE.str.contains('BLC_8') & carteira.TP_ATIVO.str.contains("Ação|Ações|Opção|Opcões|Outr|Recibo|BDR", flags=re.IGNORECASE)].DS_ATIVO.str.extract(r'([A-Z]{0,5}\d+)', expand=False))\
            .fillna(carteira[carteira.FILE.str.contains('BLC_8')].DS_ATIVO.str.extract(r'(BR[A-Z0-9]+)', expand=False))\
            .fillna(carteira[carteira.DS_ATIVO.str.lower().isin(['disponibilidades', 'valores a pagar', 'valores a receber'])].DS_ATIVO.str.upper()) \
            .fillna(carteira[carteira.TP_ATIVO == 'Título público federal'].DS_ATIVO.str.extract(r'(\d{6})', expand=False)) \
            .fillna(carteira.DS_ATIVO)\
            .fillna(carteira.TP_APLIC)
pesos = carteira.pivot_table(index='DT_COMPTC', columns=pivot_cols, values='peso', aggfunc='sum')
#pesos['VALORES A PAGAR'] *= -1

In [143]:
carteira['DENOM_SOCIAL'].unique()[0]

'OPPORTUNITY SELECTION MASTER FUNDO DE INVESTIMENTO EM AÇÕES'

In [112]:
carteira[carteira.DT_COMPTC == '2011-07-31']['VL_MERC_POS_FINAL'].sum()

147744019.1666403

In [113]:
pl[pl.DT_COMPTC == '2011-07-31']

,Id,AG_RISCO,BV_MERC,CD_ATIVO,CD_ATIVO_BV_MERC,CD_BV_MERC,CD_INDEXADOR_POSFX,CD_ISIN,CD_PAIS,CD_SELIC,...,TP_FUNDO,TP_NEGOC,TP_TITPUB,FILE,VL_AQUIS_NEGOC,VL_ATIVO_EXTERIOR,VL_CUSTO_POS_FINAL,VL_MERC_POS_FINAL,VL_PATRIM_LIQ,VL_VENDA_NEGOC
63,37160312,None,None,None,None,None,None,None,None,None,...,FI,None,None,PL,None,None,None,None,139248368.0,None


In [114]:
carteira[carteira.DT_COMPTC == '2011-06-30'][['DS_ATIVO','TP_ATIVO','VL_MERC_POS_FINAL']]

,DS_ATIVO,TP_ATIVO,VL_MERC_POS_FINAL
273,None,Fundo de Investimento e de Cotas,7.679765e+05
274,None,Fundo de Investimento e de Cotas,7.679166e+05
2384,BRASIL ON NM,Ação ordinária,1.523544e+07
2385,BRADESCO ON EJ N1,Ação ordinária,1.447539e+06
2386,BRF SA ON NM,Ação ordinária,0.000000e+00
2387,B2W DIGITAL ON NM,Ação ordinária,6.683692e+05
2388,CYRELA REALT ON ED NM,Ação ordinária,2.383081e+06
2389,EVEN ON NM,Ação ordinária,2.628004e+06
2390,CIA HERING ON NM,Ação ordinária,1.003243e+07
2391,ITAUUNIBANCO ON N1,Ação ordinária,2.076043e+06
